In [1]:
import os
import sys
import math
import logging
import structlog
from pathlib import Path
import json

import tomli
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={"figure.figsize": (12, 6.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

In [2]:
logging.basicConfig(level=logging.WARNING, stream=sys.stdout)

In [3]:
import pytanis
from pytanis import GSheetClient, PretalxClient
from pytanis.google import Scope, mark_rows
from pytanis.review import read_assignment_as_df, save_assignments_as_json, Col
from pytanis.pretalx import subs_as_df, reviews_as_df, speakers_as_df
from pytanis.utils import implode

In [4]:
# Be aware that this notebook might only run with the following version
pytanis.__version__ 

'0.4'

In [5]:
# Import event-specific settings to don't have them here in the notebook
with open('config.toml', 'rb') as fh:
    cfg = tomli.load(fh)


# Read Reviews and all Submissions

In [6]:
pretalx_client = PretalxClient(blocking=True)
subs_count, subs = pretalx_client.submissions(cfg['event_name'],  params={"questions": "all"})
spkrs_count, spkrs = pretalx_client.speakers(cfg['event_name'], params={"questions": "all"})
revs_count, revs = pretalx_client.reviews(cfg['event_name'])
subs, revs, spkrs = list(subs), list(revs), list(spkrs)

  0%|          | 0/422 [00:00<?, ?it/s]

  0%|          | 0/347 [00:00<?, ?it/s]

  0%|          | 0/1390 [00:00<?, ?it/s]

In [7]:
subs_df = subs_as_df(subs, with_questions=True)
revs_df = reviews_as_df(revs)
spkrs_df = speakers_as_df(spkrs, with_questions=True)

# Join speakers and submissions

In [8]:
# explode on speakers and their names, join on speaker code and implode it again to have one submission per row
subs_df = subs_df.explode([Col.speaker_code, Col.speaker_name])
subs_df = pd.merge(subs_df, spkrs_df.drop(columns=[Col.speaker_name, Col.submission]), on=Col.speaker_code)
subs_df = implode(subs_df, [col for col in spkrs_df if col not in [Col.submission]])

# Balance reviews by their personal mean (remove evaluation bias)

In [9]:
user_means = revs_df.groupby([Col.pretalx_user], group_keys=False)[[Col.review_score]].mean().reset_index()
revs_df["Avg Review Score"] = pd.merge(revs_df[[Col.pretalx_user]], user_means, on=Col.pretalx_user, how='left')[Col.review_score]
revs_df["Debiased Review Score"] = revs_df[Col.review_score] - revs_df["Avg Review Score"]

# Join with submissions

In [10]:
avg_scores = pd.merge(subs_df, revs_df[[Col.submission, Col.review_score, "Debiased Review Score"]], on=Col.submission, how='left')
avg_scores = avg_scores.groupby([Col.submission]).agg(**{Col.review_score: (Col.review_score, lambda x: x.tolist()),
                                                         "Avg Review Score": (Col.review_score, 'mean'), 
                                                         "Debiased Review Score": ("Debiased Review Score", lambda x: [f"{n:.2}" for n in x.tolist()]), 
                                                         "Avg Debiased Review Score": ("Debiased Review Score", "mean"), 
                                                         Col.nreviews: (Col.review_score, 'count')})

In [11]:
subs_df = pd.merge(subs_df, avg_scores, on=Col.submission)

# Get public voting results and join

In [12]:
votes_df = pd.read_csv("./pyconde-pydata-berlin-2023-public-votes.csv")
votes_df = votes_df.rename(columns={'code': Col.submission, 'score': Col.vote_score})
votes_df = votes_df.groupby(Col.submission).aggregate({Col.vote_score: lambda x: x.tolist()}).reset_index()

In [13]:
# add a few informative columns
votes_df[Col.nvotes] = votes_df[Col.vote_score].str.len()
votes_df["Votes Sum > 1"] = votes_df[Col.vote_score].map(lambda votes: sum([vote for vote in votes if vote > 1]))
votes_df["Avg Vote Score"] = votes_df[Col.vote_score].map(lambda x: np.mean(x))

In [14]:
subs_df = pd.merge(subs_df, votes_df, on=Col.submission, how='left')

# Restructure the Sheet

In [15]:
subs_df.drop(columns=['Q: Link to talk slides',
                      'Q: Twitter handle', 
                      'Q: I have read and agree to the Code of Conduct', 
                      'Q: I agree to follow the health guidelines as set out by the organizers / venue',
                      'Created',
                      'Q: Picture',
                      'Q: Public link to supporting material',
                      'Q: Abstract as a tweet',
                      'Submission type id']
            , inplace=True)

In [16]:
subs_df.rename(columns={'Q: Expected audience expertise: Python': 'Python expertise',
                        'Q: Expected audience expertise: Domain': 'Domain expetise',
                        'Q: I identify as a member of an underrepresented group': 'Underrepresented',
                        'Q: Country of residence': 'Country',
                        'Q: Github': 'Github',
                        'Q: LinkedIn': 'LinkedIn',
                        'Q: Homepage': 'Homepage',
                        'Q: Company / Institute': 'Affiliation',
                        'Q: Position / Job': 'Position',
                        'Q: I will present my talk on site': 'Onsite talk',
                        'Q: Notes for reviewers only': 'Reviewer notes'},
              inplace=True)

In [17]:
# Split track in main and subtrack
subs_df.insert(2, 'Main track', subs_df[Col.track].map(lambda x: x.split(":")[0] if not pd.isna(x) else x))
subs_df[Col.track] = subs_df[Col.track].map(lambda x: x.split(":")[-1] if not pd.isna(x) else x)

In [18]:
# Have "Pending state" second but last column
col = subs_df.pop("Pending state")
subs_df = pd.concat([subs_df, col.to_frame()], axis=1)
# Have "State" second but last column
col = subs_df.pop("State")
subs_df = pd.concat([subs_df, col.to_frame()], axis=1)

In [19]:
# avoid multi-lines cells in GSheet
subs_df['Reviewer notes'] = subs_df['Reviewer notes'].str.replace('\n', ' ')

In [20]:
subs_df.sort_values("Votes Sum > 1", inplace=True, ascending=False)
subs_df.reset_index(inplace=True, drop=True)

## Save it to GSheet

In [21]:
# make subsmission code a hyperlink
subs_df[Col.submission] = subs_df[Col.submission].map(lambda sub: f'=HYPERLINK("https://pretalx.com/orga/event/{cfg["event_name"]}/submissions/{sub}", "{sub}")')

In [22]:
gsheet_client = GSheetClient(read_only=False)

In [23]:
gsheet_client.save_df_as_gsheet(subs_df, cfg['selection_spread_id'], cfg['selection_work_name'], resize=False)

In [24]:
# do some formatting
from gspread_formatting import *

worksheet = gsheet_client.gsheet(cfg['selection_spread_id'], cfg['selection_work_name'])
set_frozen(worksheet, rows=1, cols=1);

In [25]:
mask = (subs_df["State"] == 'rejected') | (subs_df["State"] == 'withdrawn') | (subs_df["State"] == 'canceled')
mark_rows(worksheet, mask, 'firebrick')

In [26]:
mask = (subs_df["State"] == 'confirmed')
mark_rows(worksheet, mask, 'green')

In [27]:
mask = (subs_df["State"] == 'accepted')
mark_rows(worksheet, mask, 'limegreen')

In [28]:
mask = (subs_df["Pending state"] == 'rejected')
mark_rows(worksheet, mask, 'lightcoral')

In [29]:
mask = (subs_df["Pending state"] == 'accepted')
mark_rows(worksheet, mask, 'greenyellow')

## Confirmed Talks

In [30]:
conf_df = subs_df.loc[subs_df['State'] == 'confirmed']

In [31]:
gsheet_client.save_df_as_gsheet(conf_df, cfg['selection_spread_id'], cfg['confirmation_work_name'])

In [32]:
worksheet = gsheet_client.gsheet(cfg['selection_spread_id'], cfg['confirmation_work_name'])
set_frozen(worksheet, rows=1, cols=1);